In [1]:
from core import models
from core import data 
import utils 
import torch.nn as nn
import torch
import os
from tqdm import trange, tqdm


In [2]:
data_dir = "../../different_cards"
device = "cpu"
exp_dir = "../../classifier_test"

utils.prepare_directory(exp_dir)

directory: ../../classifier_test already exists, backing up this folder ... 
backup directory also exists, removing the backup directory first


In [3]:
logger_titles = [
        'lr',
        'train_loss',
        'val_loss',
        'train_top1',
        'val_top1',
        'train_time',
        'val_time']


In [4]:
criterion = nn.CrossEntropyLoss().to(device)
datahandler = data.CardDataHandler(data_dir, uniform_sampling=True)

13 validation split from training
115 training remains


In [5]:
trainloader, valloader = datahandler.get_dataloaders(batch_size=8)

created uniform sampler


In [6]:
# create the convnet and optimizer
model = models.ConvNet()

lr = 0.0001
momentum = 0.9
wd = 1e-4
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=wd)

In [7]:
log_dir = os.path.join(exp_dir, "training_logs")
utils.prepare_directory(log_dir)
logger = utils.Logger(log_dir, 'logs', logger_titles)

In [8]:
#training loop
n_epochs = 10

best_top1 = 0 
best_confmat = None 
for epoch in trange(0, n_epochs, desc="epoch_monitor", dynamic_ncols=False):
    train_loss, train_top1, train_time = utils.train(model, trainloader, optimizer, criterion, device)
    val_loss, val_top1, confmat, val_time = utils.evaluate(model, valloader, criterion, device)
    
    is_best = best_top1 < val_top1
    if is_best:
        best_top1 = val_top1
        best_confmat = confmat
        
    utils.save_checkpoint({
        'model': model.state_dict(),
        'epoch': epoch, 
        'best_top1':best_top1,
        'val_top1':val_top1}, 
        exp_dir,
        is_best)
    
    logger.append(
            [lr, train_loss, val_loss,
            train_top1, val_top1,  
            train_time, val_time],
            epoch)

    print("train loss: {:.2f}  val loss: {:.2f} val acc: {:.2f}".format(train_loss, val_loss, val_top1))

logger.close()
    

training loop:   0%|          | 0/115 [00:00<?, ?it/s]/Users/jasonzhang/opt/anaconda3/envs/taskTohsakaTorch/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1623459044803/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)

training loop: 100%|██████████| 115/115 [00:25<00:00,  4.44it/s]

epoch_monitor:  10%|█         | 1/10 [00:39<05:55, 39.55s/it]

train loss: 1.14  val loss: 1.04 val acc: 0.38



training loop: 100%|██████████| 115/115 [00:25<00:00,  4.54it/s]

epoch_monitor:  20%|██        | 2/10 [01:18<05:13, 39.24s/it]

train loss: 0.98  val loss: 0.92 val acc: 0.62



training loop: 100%|██████████| 115/115 [00:24<00:00,  4.73it/s]

epoch_monitor:  30%|███       | 3/10 [01:56<04:30, 38.61s/it]

...

train loss: 0.32  val loss: 0.32 val acc: 1.00



training loop: 100%|██████████| 115/115 [00:21<00:00,  5.25it/s]

epoch_monitor:  90%|█████████ | 9/10 [05:29<00:35, 35.46s/it]

train loss: 0.30  val loss: 0.27 val acc: 1.00



training loop: 100%|██████████| 115/115 [00:21<00:00,  5.30it/s]

epoch_monitor: 100%|██████████| 10/10 [06:04<00:00, 36.43s/it]


train loss: 0.25  val loss: 0.26 val acc: 1.00


In [9]:
confmat

array([[4, 0, 0],
       [0, 2, 0],
       [0, 0, 7]])

In [10]:
train_top1

1.0